In [1]:
from sklearn.tree import DecisionTreeRegressor

In [2]:
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
import pylab as plot

In [3]:
train = pd.read_csv('./datas/train.csv')

In [4]:
dong = pd.read_csv("./datas/dongrank.csv")

In [5]:
train = pd.merge(train, dong, on=["dong","city"])

In [6]:
train.rename(columns={'index':'dong3'},inplace=True)
train['P/m^2'] = train['transaction_real_price']/train['exclusive_use_area'] # 평당가격 column 추가
train['year'] =  round(train['transaction_year_month'] / 100)
train['month'] = train['transaction_year_month'] % 100

train = train[['transaction_id', 'apartment_id', 'city',  'dong', 'jibun', 'apt',
               'addr_kr', 'exclusive_use_area', 'year_of_completion',
               'transaction_year_month','year', 'month', 'transaction_date', 'floor', 'P/m^2',
               'transaction_real_price',"dong3"]]

In [7]:
train["dong2"] = train["dong3"].astype('category')
train["transaction_id2"] = train["transaction_id"].astype('category')
train["apartment_id2"] = train["apartment_id"].astype('category')

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1216553 entries, 0 to 1216552
Data columns (total 20 columns):
 #   Column                  Non-Null Count    Dtype   
---  ------                  --------------    -----   
 0   transaction_id          1216553 non-null  int64   
 1   apartment_id            1216553 non-null  int64   
 2   city                    1216553 non-null  object  
 3   dong                    1216553 non-null  object  
 4   jibun                   1216553 non-null  object  
 5   apt                     1216553 non-null  object  
 6   addr_kr                 1216553 non-null  object  
 7   exclusive_use_area      1216553 non-null  float64 
 8   year_of_completion      1216553 non-null  int64   
 9   transaction_year_month  1216553 non-null  int64   
 10  year                    1216553 non-null  float64 
 11  month                   1216553 non-null  int64   
 12  transaction_date        1216553 non-null  object  
 13  floor                   1216553 non-null  

In [9]:
train.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,year,month,transaction_date,floor,P/m^2,transaction_real_price,dong3,dong2,transaction_id2,apartment_id2
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,2008.0,1,21~31,2,442.112709,37500,354,354,0,7622
1,98,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,200803,2008.0,3,11~20,2,463.334119,39300,354,354,98,7623
2,511,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200903,2009.0,3,21~31,3,473.944824,40200,354,354,511,7622
3,840,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),81.02,2002,200909,2009.0,9,1~10,1,527.647494,42750,354,354,840,7623
4,1201,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,201011,2010.0,11,21~30,3,609.526055,51700,354,354,1201,7623


In [17]:
train_x = train[["dong2","apartment_id2","exclusive_use_area","year_of_completion","year","month","floor"]]
train_y = train[["transaction_real_price"]]

test_x = train[["dong2","apartment_id2","exclusive_use_area","year_of_completion","year","month","floor"]]
test_y = train[["transaction_real_price"]]

In [18]:
train

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,year,month,transaction_date,floor,P/m^2,transaction_real_price,dong3,dong2,transaction_id2,apartment_id2
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.8200,2002,200801,2008.0,1,21~31,2,442.112709,37500,354,354,0,7622
1,98,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.8200,2002,200803,2008.0,3,11~20,2,463.334119,39300,354,354,98,7623
2,511,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.8200,2002,200903,2009.0,3,21~31,3,473.944824,40200,354,354,511,7622
3,840,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),81.0200,2002,200909,2009.0,9,1~10,1,527.647494,42750,354,354,840,7623
4,1201,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.8200,2002,201011,2010.0,11,21~30,3,609.526055,51700,354,354,1201,7623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216548,1226761,8987,부산광역시,녹산동,262-1,유나베네스1차,녹산동 262-1 유나베네스1차,62.2475,2017,201710,2017.0,10,21~31,5,305.233142,19000,105,105,1226761,8987
1216549,1226762,8987,부산광역시,녹산동,262-1,유나베네스1차,녹산동 262-1 유나베네스1차,62.2475,2017,201710,2017.0,10,21~31,2,302.020161,18800,105,105,1226762,8987
1216550,1226763,8987,부산광역시,녹산동,262-1,유나베네스1차,녹산동 262-1 유나베네스1차,72.5242,2017,201710,2017.0,10,21~31,2,286.800820,20800,105,105,1226763,8987
1216551,1226764,8987,부산광역시,녹산동,262-1,유나베네스1차,녹산동 262-1 유나베네스1차,72.5242,2017,201710,2017.0,10,21~31,11,289.558520,21000,105,105,1226764,8987


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
x = train[["dong2","apartment_id2","exclusive_use_area","year_of_completion","year","month","floor"]]
y = train[["transaction_real_price"]]

In [30]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2,
                                                   random_state=13)

#max_depth_list = []

model = RandomForestRegressor(n_estimators=100,
                              random_state=13)
model.fit(train_x, train_y)
pred_train = model.predict(train_x)
pred_test = model.predict(test_x)

/Users/ijiyeong/opt/anaconda3/envs/jiyoung/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [31]:
mse = mean_squared_error(test_y, pred_test)
r2 = r2_score(test_y, pred_test)
rmse = mean_squared_error(test_y, pred_test)**0.5

In [32]:
print('Mean squared error: ', mse)
print('R2 score: ', r2)
print('RMSE score:', rmse)

Mean squared error:  15527867.114304654
R2 score:  0.9838604982620426
RMSE score: 3940.541474760119
